# **Introduction 🌍**

Welcome to this notebook on **Efficient Loading of Raster and Vector Data in Wherobots**. This guide is designed for users working in a hosted notebook environment with preconfigured tools and libraries, making it easier to focus on geospatial workflows without needing extensive setup.

### Objectives 🎯

In this notebook, you will:

1. 🗺️ Understand how to work with vector and raster data types in Wherobots.
2. ☁️ Learn how to load data from cloud storage and hosted environments.
3. 📊 Prepare to manage geospatial datasets efficiently in the Wherobots ecosystem.

By the end of this tutorial, you will have foundational knowledge to load, query, and manage geospatial data in Wherobots using a combination of **Apache Sedona** and **WherobotsDB**.

### What is Wherobots? 🤖

Wherobots is a **cloud-native spatial analytics platform** designed for large-scale geospatial data processing. 🚀 It offers:

- **WherobotsDB**: A scalable geospatial database based on Apache Iceberg, but for gerospatial.
- **Apache Sedona Integration**: High-performance library geospatial queries and visualizations.
- **Out-of-Database Raster Support**: Handle massive raster datasets efficiently.
- **GeoParquet Support**: Work with optimized, open-source vector data formats.

---

### Why Use a Hosted Environment? 🖥️

A hosted notebook environment in Wherobots provides:

- **Preconfigured Libraries**: Tools like Apache Sedona and the WherobotsDB compute environment are ready to use.
- **Managed Authentication**: Seamless connection to Wherobots Cloud without manual setup.
- **Integrated Storage**: Access to both managed storage and external S3 buckets for geospatial data.
- **Scalability**: Optimized runtimes for small-scale experimentation to large-scale production workflows.

---

### Key Concepts 📚

#### Vector Data
- Represents discrete features like points, lines, and polygons.
- Common formats:
  - **GeoParquet**: Optimized for modern geospatial workflows.
  - **Shapefile**: Legacy format for geospatial data.
  - **GeoJSON**: Lightweight and human-readable.

#### Raster Data
- Represents continuous phenomena using a grid of cells (e.g., elevation, satellite imagery).
- Common formats:
  - **Cloud-Optimized GeoTIFF (COG)**: Designed for efficient cloud storage and access.
  - **NetCDF**: Often used for multidimensional climate data.

#### GeoParquet
GeoParquet is an open-source format designed for modern vector data workflows:

- **Advantages**:
  - Highly compact and optimized.
  - Native support for spatial indexing.
  - Compatibility with large-scale distributed processing frameworks.

---

In the following sections, we will:
1. Set up our hosted environment.
2. Load vector data into Wherobots.
3. Load raster data into Wherobots.
4. Write both vector and raster data back to GeoParquet and cloud storage.

# **Section 2: Preconfigured Environment Setup 🚀**

## **Introduction**
Before working with geospatial data in **Wherobots**, we need to initialize our **Sedona environment** and set up a **connection to S3 storage**. These steps ensure that our data processing workflows run efficiently within the **hosted notebook environment**.

In this section, we will:
1. **Initialize Apache Sedona** – The geospatial engine that powers spatial operations in Spark.
2. **Connect to S3 storage** – The cloud storage system where our geospatial datasets are stored.

---

## **1️⃣ Initializing Apache Sedona**
**Apache Sedona** is a spatial computing extension for **Apache Spark**. It allows us to efficiently load, transform, and analyze vector and raster data in **distributed computing environments**.

### **🔹 Why do we need Sedona?**
- Provides **Spatial SQL** capabilities (e.g., `ST_Intersects`, `ST_Within`).
- Supports **vector** and **raster** data processing at scale.
- Works seamlessly with **GeoParquet**, **GeoTIFF**, and other geospatial formats.

### **🔹 Initializing Sedona in the Hosted Notebook**
The following code initializes Sedona in our environment:

```python
# Import the Sedona library
from sedona.spark import SedonaContext

# Create the Sedona context (auto-configured for Wherobots)
sedona = SedonaContext.builder().getOrCreate()
```

#### **🛠️ What’s Happening?**
- `SedonaContext.builder()` **automatically detects** the Spark environment.
- `.getOrCreate()` **ensures only one instance** of Sedona is created.
- This setup **enables spatial functions** within Spark SQL.

---

## **2️⃣ Connecting to S3 Storage**
Most geospatial datasets are **too large** to store locally, so we use **Amazon S3 (Simple Storage Service)** to manage and access spatial data efficiently.

### **🔹 Why use S3 for geospatial data?**
✅ Stores **large-scale** vector and raster datasets.  
✅ Enables **cloud-based querying** without local downloads.  
✅ Supports **Out-of-Database (Out-DB) rasters** for efficient processing.

### **🔹 Verifying the S3 Connection**
Let’s test if we can **list files** in an S3 bucket:

```python
# Example: List files in an S3 bucket
s3_files = sedona.spark.read.format("binaryFile").load("s3a://wherobots-public-data/overturemaps/")
s3_files.show(5, truncate=False)
```

🔍 **This command helps us:**
- Verify our connection to **Wherobots' public S3 bucket** for the data in this tutorial.
- Confirm that we can access spatial datasets stored in the cloud.

---

## **✅ Summary**
- **Apache Sedona** is initialized to enable spatial computing in Spark.
- **S3 Storage** is configured for reading **vector and raster** data from the cloud.
- We verified the setup by **listing files** from an S3 bucket.

With this setup, we are now ready to **load vector and raster data** into our notebook! 🚀🌍

# ⌨️ **Section 2: Code**

In [ ]:
# Step 1: Import necessary libraries and connect Apache Sedona to the Wherobots runtime
from sedona.spark import SedonaContext

config = SedonaContext.builder() \
    .getOrCreate()

sedona = SedonaContext.create(config)

In [ ]:
# Now read out all the files we will be looking at in the tutorial 

from pyspark.sql.functions import input_file_name

s3_path = 's3a://wherobots-examples/data/onboarding_1/'

try:
    # List files in the S3 bucket (without loading full contents)
    s3_files = sedona.read.format("binaryFile").load(s3_path).select(input_file_name().alias("file_name"))
    # Show only file names
    print(f"Files in {s3_path}:")
    s3_files.show(truncate=False)
    
except Exception as e:
    print(f"Error accessing S3 path: {e}")

# **Section 3: Loading Vector Data 🗺️**

## **Introduction**
**Vector data** represents real-world features using **points, lines, and polygons**. In Wherobots, we can load vector datasets from various formats, including **GeoParquet, GeoJSON, Shapefile, and CSV with geometry columns**.

In this section, we will:
1. **Understand vector data types and formats** in Wherobots.
2. **Load vector datasets** from different sources like S3 and local storage.
3. **Create a temporary view** to enable **SQL-based** spatial queries.

---

## **1️⃣ Understanding Vector Data Formats**
Vector data is stored in multiple formats, each optimized for different use cases. Below are some common formats **supported in Wherobots**:

| **Format**    | **Description** |
|--------------|----------------|
| **GeoParquet** 🏗️ | A modern, efficient format for vector data that supports indexing and partitioning. |
| **GeoJSON** 📜 | A lightweight, human-readable format for spatial data on the web. |
| **Shapefile** 📂 | A legacy format widely used in GIS applications (requires multiple files). |
| **CSV with Geometry** 📊 | Tabular data containing WKT (Well-Known Text) geometries. |

### **🛠️ Why Use GeoParquet?**
✅ **Optimized for big data processing** (columnar format).  
✅ **Supports partitioning and indexing** (faster spatial queries).  
✅ **Seamless compatibility** with modern data lakes and cloud storage.  

---

## **2️⃣ Loading Vector Data from S3**
Wherobots provides **pre-configured access** to **public datasets** stored in **S3 buckets**. Let’s load a **GeoParquet** file from S3:

```python
# Load vector data from S3 (GeoParquet format)
vector_df = sedona.read.format("geoparquet").load("s3a://wherobots-public-data/overturemaps/theme=buildings/type=building")

# Print the schema to inspect the structure of the data
vector_df.printSchema()

# Display the first few rows of vector data
vector_df.show(5, truncate=False)
```

#### **🛠️ What’s Happening?**
- **`format("geoparquet")`** → Specifies that we are reading a **GeoParquet** file.
- **`load("s3a://...")`** → Loads the dataset **directly from S3** without downloading it locally.
- **`.printSchema()`** → Displays the **columns** in our dataset.
- **`.show(5, truncate=False)`** → Displays the first **five rows** of vector data.

---

## **3️⃣ Loading GeoJSON and Other Vector Formats**
In addition to **GeoParquet**, we can load **GeoJSON, Shapefile, and CSV** vector datasets.

### **🔹 Loading GeoJSON from Local Storage**
GeoJSON is often used for **web-based mapping applications**.

```python
# Load a GeoJSON file from local managed storage
geojson_df = sedona.read.format("json").load("/data/shared/sample.geojson")

# Show first few rows
geojson_df.show(5, truncate=False)
```

---

### **🔹 Loading a Shapefile**
Shapefiles consist of **multiple files** (`.shp`, `.dbf`, `.shx`), so we load **the directory containing them**.

```python
# Load a shapefile from local storage
shapefile_df = sedona.read.format("shapefile").load("/data/shared/shapefile_folder/")

# Show first few rows
shapefile_df.show(5, truncate=False)
```

---

### **🔹 Loading a CSV with a Geometry Column**
CSV files can store **spatial data** using **WKT (Well-Known Text) geometries**.

```python
from pyspark.sql.functions import expr

# Load CSV data with a geometry column
csv_df = sedona.read.format("csv").option("header", "true").load("/data/shared/sample.csv")

# Convert WKT column into a proper geometry column
vector_df = csv_df.withColumn("geometry", expr("ST_GeomFromWKT(geometry_column)"))

# Show first few rows
vector_df.show(5, truncate=False)
```

#### **🛠️ What’s Happening?**
- **`option("header", "true")`** → Ensures the first row is treated as column names.
- **`ST_GeomFromWKT(geometry_column)`** → Converts **WKT text** into a proper **geometry object**.

---

## **4️⃣ Creating a Temporary SQL View**
Once vector data is loaded, we **register it as a temporary view** so we can **query it using SQL**.

```python
# Register vector data as a temporary SQL view
vector_df.createOrReplaceTempView("vector_data")
```

### **🔹 Running a Spatial Query**
We can now perform **SQL-based spatial analysis**:

```sql
-- Query all vector data where geometry intersects with a given polygon
SELECT * FROM vector_data
WHERE ST_Intersects(geometry, ST_GeomFromText('POLYGON((-122.5 37.5, -122.5 37.6, -122.4 37.6, -122.4 37.5, -122.5 37.5))'))
```

#### **🛠️ What’s Happening?**
- **`ST_Intersects(geometry, ST_GeomFromText(...))`** → Checks if vector features **intersect** with a given **polygon**.
- **This enables powerful spatial filtering** within large datasets.

---

## **✅ Summary**
- **Vector data** represents **points, lines, and polygons**.
- We loaded vector datasets from **S3 and local storage** in formats like **GeoParquet, GeoJSON, Shapefile, and CSV**.
- We **converted WKT strings** into proper geometries for spatial queries.
- Finally, we **registered our data as a SQL view** and ran **a spatial query**.

# ⌨️ **Section 3: Code**

In [ ]:
geo_parquet_path = 's3://wherobots-examples/data/onboarding_1/nyc_buildings.parquet'

# Load GeoParquet data into a Spark DataFrame
vector_df = sedona.read.format("geoparquet").load(geo_parquet_path)

In [ ]:
vector_df.printSchema()

In [ ]:
vector_df.show(5)

In [ ]:
# Define the local path for GeoJSON data
geojson_path = "s3://wherobots-examples/data/onboarding_2/nyc_neighborhoods.geojson"

# Load GeoJSON data into a Spark DataFrame
print("Loading GeoJSON data...")
geojson_df = sedona.read.format("geojson").load(geojson_path)

In [ ]:
geojson_df.printSchema()

In [ ]:
from pyspark.sql import functions as f 

geojson_df = sedona.read.format("geojson") \
    .load(geojson_path) \
    .withColumn("borough", f.expr("properties['borough']")) \
    .withColumn("boroughCode", f.expr("properties['boroughCode']")) \
    .withColumn("neighborhood", f.expr("properties['neighborhood']")) \
    .drop("_corrupt_record") \
    .drop("properties") \
    .drop("type") 

In [ ]:
geojson_df.printSchema()

In [ ]:
print("Converting Lat/Long decimals in CSV data to geometry column...")
csv_path = "s3://wherobots-examples/data/onboarding_2/311_Service_Requests_from_2010_to_Present_20240912.csv"
csv_df = sedona.read.format("csv").option("header", "true").load(csv_path)

In [ ]:
csv_df.printSchema()

In [ ]:
csv_df = sedona.read.format("csv") \
    .option("header", "true") \
    .load(csv_path) \
    .withColumn("geometry", f.expr("ST_MakePoint(Longitude, Latitude, 4326)"))

In [ ]:
csv_df.createOrReplaceTempView('csv_df')

In [ ]:
# Example 1: Filter points within a polygon
polygon_wkt = "POLYGON((-73.9945201121 40.7512166031, -73.9925054739 40.7512166031, -73.9925054739 40.7498572827, -73.9945201121 40.7498572827, -73.9945201121 40.7512166031))"
query = f"""
SELECT * FROM csv_df
WHERE ST_Intersects(geometry, ST_GeomFromText('{polygon_wkt}'))
"""
filtered_df = sedona.sql(query)

In [ ]:
%%time
# Show results
print(f"Number of filtered points within polygon: {filtered_df.count()}")


# **Section 4: Loading Raster Data 🛰️**

## **Introduction**
Raster data represents **continuous spatial information** such as:
- Satellite imagery 🛰️
- Elevation models ⛰️
- Climate data 🌦️

Unlike vector data, which consists of **points, lines, and polygons**, raster data is stored as a **grid of pixels**, where each pixel represents a value (e.g., temperature, elevation).

In this section, we will:
1. **Understand raster data formats** supported in Wherobots.
2. **Load raster datasets from S3 and local storage**.
3. **Process raster data efficiently** using tiling and querying techniques.

---

## **1️⃣ Understanding Raster Data Formats**
Raster datasets come in various formats, optimized for different workflows.

| **Format**    | **Description** |
|--------------|----------------|
| **GeoTIFF** 🏞️ | A widely used raster format for geospatial imagery. |
| **Cloud-Optimized GeoTIFF (COG)** ☁️ | A version of GeoTIFF optimized for fast cloud access. |
| **NetCDF** 🌍 | Commonly used for scientific climate and weather data. |
| **JPEG2000** 🖼️ | A compressed raster format with high quality. |
| **HDF (Hierarchical Data Format)** 📦 | Used for large datasets in Earth science. |

### **🔹 Why Use Cloud-Optimized GeoTIFF (COG)?**
✅ **Faster access in cloud storage** (only reads necessary parts of the file).  
✅ **Optimized for parallel processing** in big data environments.  
✅ **Compatible with most GIS tools** like QGIS, GDAL, and Wherobots.  

---

## **2️⃣ Loading Raster Data from S3**
Now that we have **verified the available files**, we can **load a Cloud-Optimized GeoTIFF (COG) from S3**.

```python
# Load a Cloud-Optimized GeoTIFF (COG) from S3
raster_df = sedona.read.format("raster").load("s3a://wherobots-public-data/satellite_imagery/sample.tif")

# Print schema
raster_df.printSchema()

# Show raster metadata
raster_df.show(5, truncate=False)
```

### **🛠️ What’s Happening?**
- **`format("raster")`** → Loads the raw file **without reading** all pixel values.
- **`RS_FromGeoTiff(content)`** → Converts raster content into a **structured raster object**.
- **`printSchema()`** → Displays metadata about the raster dataset.

---

## **3️⃣ Loading Raster Data from Local Storage**
If the raster dataset is stored in **local managed storage**, we can load it directly.

```python
# Load raster from local storage
local_raster_df = sedona.read.format("raster").load("/data/shared/sample.tif")

# Show raster metadata
local_raster_df.show(5, truncate=False)
```

### **🛠️ What’s Happening?**
- The **same method** is used as with S3, but the path is a local file (`/data/shared/...`).
- The dataset is stored **within the Wherobots notebook environment**.

---

## **4️⃣ Optimizing Raster Data with Tiling**
Large raster files **must be tiled** to improve performance.

```python
# Explode raster into tiles
tiled_raster_df = raster_df.selectExpr("RS_Explode(rast) as tiles")

# Create a SQL view for tiled rasters
tiled_raster_df.createOrReplaceTempView("tiled_raster_df")

# Show some tiles
tiled_raster_df.show(5, truncate=False)
```

### **🛠️ What’s Happening?**
- **`RS_Explode(rast)`** → Breaks the raster into **smaller, more manageable tiles**.
- **Why?** → Tiled rasters allow us to **query specific areas** without reading the entire dataset.

---

## **5️⃣ Querying Raster Data**
We can **extract pixel values** and perform **spatial queries** on raster datasets.

### **Extracting Pixel Values at Specific Coordinates**
```sql
-- Query pixel value at a specific coordinate
SELECT RS_PixelAsPoint(rast, 10, 15) AS pixel_point FROM raster_df;
```
🔹 Returns the **geographic coordinates** of a specific pixel.

---

### **Performing a Spatial Range Query**
```sql
-- Select rasters that intersect with a given polygon
SELECT rast 
FROM raster_df 
WHERE RS_Intersects(rast, ST_GeomFromText('POLYGON((-122.5 37.5, -122.5 37.6, -122.4 37.6, -122.4 37.5, -122.5 37.5))'));
```
🔹 Returns only the **raster tiles** that intersect with the polygon.

---

## **✅ Summary**
- **Raster data** represents continuous geographic information.
- We **listed available raster files** in an S3 bucket.
- We **loaded Cloud-Optimized GeoTIFF (COG) files** into Wherobots.
- We **converted raw files into structured raster objects**.
- We **tiled raster data** for efficient querying.
- We **ran spatial SQL queries** to extract pixel values.


# ⌨️ **Section 4 Code**

In [ ]:
# Define the file path for a sample raster dataset
raster_file = "s3a://io-10m-annual-lulc/15T_2023.tif"

In [ ]:
# Load a Cloud-Optimized GeoTIFF (COG) from S3
raster_df = sedona.read.format("raster").load(raster_file)

In [ ]:
raster_df.show()

In [ ]:
raster_df.createOrReplaceTempView('raster_df')

In [ ]:
raster_df_tiled = raster_df

Below is the **point location** we are querying against the **raster dataframe**:

![Query Area](https://i.ibb.co/W4K0Lg90/Clean-Shot-2025-02-05-at-12-46-59-2x.png)  

In [ ]:
# Querying pixel value using SQL
query = """
SELECT RS_Value(rast, 
    ST_Transform(
        ST_SetSRID(
            ST_Point(-93.367556, 44.231003), 
        4326),
    'epsg:4326', 'epsg:32615')
) AS pixel_point FROM raster_df 
"""

result_df = sedona.sql(query)
result_df.where("pixel_point is not null").show(truncate=False)